# Layout Parser - Model trained


The model used here has been trained on BNF (Gallica)'s newspapers pages. they has been annotated and these annotations exported on COCO format.

It can be used to extract the images the newspapers pages contained.

The only thing to change is the paths to:
- the model & config file (.pth & .yaml)
- the pages you want to extract the pictures

This notebook is made to be used linked to your Google Drive account. You can easily adapt it to use it with Jupyter.


In [ ]:
# Install necessary packages
!pip install -q requests
!pip install -q --upgrade pip
!pip install -q opencv-python
!pip install -q torchvision
!pip install Pillow==9.5.0

# Import necessary libraries
import shutil
import requests
import glob
import re
import math
import json
import os
import cv2
from PIL import Image

# Install and import layoutparser and detectron2
#!pip install -q 'git+https://github.com/facebookresearch/detectron2.git@v0.4#egg=detectron2'
!pip install -q -U layoutparser

import layoutparser as lp

# Import PyTorch and empty the CUDA cache
import torch
torch.cuda.empty_cache()

# Print the PyTorch version
print(torch.__version__)

import requests

# Define the Hugging Face repository and model files
repo_id = "mgiardinetti/layout-parser-newspapers"
config_file = "config.yaml"
model_file = "model_final.pth"

# Create a directory to save the model
save_directory = "/content/layout-parser-newspapers"
os.makedirs(save_directory, exist_ok=True)

# Download the configuration file
config_url = f"https://huggingface.co/{repo_id}/resolve/main/{config_file}"
config_path = os.path.join(save_directory, config_file)
response = requests.get(config_url)
with open(config_path, 'wb') as f:
    f.write(response.content)

# Download the model weights
model_url = f"https://huggingface.co/{repo_id}/resolve/main/{model_file}"
model_path = os.path.join(save_directory, model_file)
response = requests.get(model_url)
with open(model_path, 'wb') as f:
    f.write(response.content)

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import layoutparser as lp

# Model and configuration file paths
model = "/content/layout-parser-newspapers/model_final.pth"
config = "/content/layout-parser-newspapers/config.yaml"

# Association id:nom for each class to detect
labels = "0:illustration 1:legende"

# Parse labels
dictlabels = {}
if labels:
    for asso in labels.split(" "):
        key, value = asso.split(":")
        dictlabels[int(key)] = value

# Load the model
try:
    if dictlabels:
        model = lp.Detectron2LayoutModel(
            config_path=config,
            model_path=model,
            label_map=dictlabels
        )
    else:
        model = lp.Detectron2LayoutModel(
            config_path=config,
            model_path=model
        )
    print("Modèle importé")
except Exception as e:
    print(f"Le modèle n'a pas été importé: {e}")


In [ ]:
import os
import glob
import re
import cv2
from PIL import Image, UnidentifiedImageError
import layoutparser as lp

# Define directories
#@markdown Indiquer les chemins vers

pages_directory = '/content/drive/My Drive/LP/similaires_rol/' # @param {type:"string"}
extractions_path = '/content/drive/My Drive/LP/' # @param {type:"string"}


seuil_score = 0.95 # @param {type:"string"}

# Process each page
for page_path in glob.glob(os.path.join(pages_directory, '*.jpg')):
    page_name = os.path.basename(page_path)

    # Read and convert image
    image = cv2.imread(page_path)
    image = image[..., ::-1]

    # Detect layout
    layout = model.detect(image)
    layout = [x for x in layout if x.score > seuil_score]
    print("_________________Extraction of the pictures_________________")

    # Extract bounding box coordinates
    layout_str = str(layout)
    x1 = list(map(int, re.findall(r'x_1=(\d+)', layout_str)))
    y1 = list(map(int, re.findall(r'y_1=(\d+)', layout_str)))
    x2 = list(map(int, re.findall(r'x_2=(\d+)', layout_str)))
    y2 = list(map(int, re.findall(r'y_2=(\d+)', layout_str)))

    # Crop and save images
    for i, (x1_i, y1_i, x2_i, y2_i) in enumerate(zip(x1, y1, x2, y2)):
                img = Image.open(page_path)
                cropped = img.crop((x1_i, y1_i, x2_i, y2_i))
                cropped_file_name = f"{os.path.splitext(page_name)[0]}_{i+1:02d}.jpg"
                cropped.save(os.path.join(extractions_path, cropped_file_name))
